In [ ]:

"""
Created on Tue Mar  7 10:55:33 2023

@author: gunahn
"""

#%%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from scipy import stats
import os
# read in data
def combine_Behavior_signal(beh_path,signal_path):

    #Behavior = pd.read_csv('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Behavior_0417/1/Day_14_2ch_GFAP_D_2022-11-15T10_56_41.csv')
    #signal = pd.read_csv('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Ref_corrected_0417/1/Day_14_BFPD_GFAP_D_Acq_221115_0001.csv')
    
    Behavior = pd.read_csv(beh_path)
    signal = pd.read_csv(signal_path)
    
    signal = signal.drop(columns=['Unnamed: 0'])
    Behavior = Behavior.drop(columns=['Unnamed: 0'])
    # change column name
    Behavior = Behavior.rename(columns={'Timestamp': 'time'})
    signal = signal.rename(columns={'index_1': 'time'})

    #%%
    # Timestamp to float time
    Behavior['time'] = Behavior['time'].astype('datetime64')
    signal['time'] = signal['time'].astype('datetime64')
    
    # calculating relative time
    signal['relative time'] =  signal['time'] - signal['time'][0]
    Behavior['relative time'] =  Behavior['time'] - Behavior['time'][0]
    
    signal['relative_time'] = signal['relative time']/ np.timedelta64(1, 's') 
    Behavior['relative_time'] = Behavior['relative time']/ np.timedelta64(1, 's')
    
    signal['reversetime'] = signal['time'] - signal['time'][len(signal)-1]
    Behavior['reversetime'] = Behavior['time'] - Behavior['time'][len(Behavior)-1]
    signal['reverse_time'] = signal['reversetime']/ np.timedelta64(1, 's') 
    Behavior['reverse_time'] = Behavior['reversetime']/ np.timedelta64(1, 's')
    # Find the first 1 for "TTL_initiation" in signal data
    x = signal.loc[signal['TTL_initiation'] == 1].head(1)
    # starting signal with the first 1 for "TTL_initiation" in signal data
    signal = signal[x.index[0]:]
    # reset index for signal
    signal = signal.reset_index(drop=True)
    # check if there are points that TTL_initiation and TTL_turn is 0 
    if len(signal[np.logical_and(signal.TTL_initiation==0, signal.TTL_turn==0)]) !=0:
        print(signal[np.logical_and(signal.TTL_initiation==0, signal.TTL_turn==0)].index)
        y= list(signal[np.logical_and(signal.TTL_initiation==0, signal.TTL_turn==0)].index)[-1]
        signal = signal[y+1:]
        signal = signal.reset_index(drop=True)
    #Merged_data = pd.merge_asof(Behavior, signal, on = 'relative time', direction='nearest')
    #%%
    
    Behavior['Trial On']= Behavior['Trial On']+1-1
    Behavior['Trial_diff'] = Behavior['Trial On'].diff()
    Behavior['In Turn Area'] = Behavior['In Turn Area']+1-1
    Behavior['Turn_diff'] = Behavior['In Turn Area'].diff()
    
    change_signal = signal[signal.TTL_initiation.diff()==-1]
    change_signal_turn = signal[signal.TTL_turn.diff()==-1]
    
    change_behavior = Behavior[Behavior.Trial_diff ==1]
    change_behavior_turn = Behavior[Behavior.Turn_diff ==1]
    
    change_signal['time_diff']= change_signal.relative_time.diff()
    change_behavior['time_diff']= change_behavior.relative_time.diff()
    useful_feature = ['relative_time', 'time_diff']
    
    diffrence = len(change_behavior)-len(change_signal)

    if diffrence == 0: 
        Differnece_between_two =change_behavior['relative_time'].values -change_signal['relative_time'].values
    else:
        change_signal = change_signal[:diffrence]
        Differnece_between_two =change_behavior['relative_time'].values -change_signal['relative_time'].values
    
    hist, edges = np.histogram(Differnece_between_two)
    y = edges[np.argmax(hist)]
    lag = round(y, 1)
    print(lag)
    # shift the signal time by lag
    
    signal['relative_time'] = signal['relative_time'] + lag
    
    # cut the last part of signal (after last initiation)
    signal = signal[:change_signal.index[-1]]
    # reset index for signal
    signal = signal.reset_index(drop=True)
    # cut the last part of behavior (after last initiation)
    Behavior = Behavior[:change_behavior.index[-1]]
    # reset index for Behavior
    Behavior = Behavior.reset_index(drop=True)
    
    change_signal = signal[signal.TTL_initiation.diff()==-1]
    change_signal_turn = signal[signal.TTL_turn.diff()==-1]
    
    change_behavior = Behavior[Behavior.Trial_diff ==1]
    change_behavior_turn = Behavior[Behavior.Turn_diff ==1]
    
    change_signal['time_diff']= change_signal.relative_time.diff()
    change_behavior['time_diff']= change_behavior.relative_time.diff()
    useful_feature = ['relative_time', 'time_diff']
    
    print(change_signal[useful_feature])
    print(change_behavior[useful_feature])
    
    Merged_data = pd.merge_asof(Behavior, signal, on='relative_time', direction='nearest')

    return diffrence, Differnece_between_two, change_signal, change_behavior, signal, Behavior, Merged_data

## Find something does not work with the above code 

In [ ]:
not_working = []
not_working_sig = []
for animal_number in range(1,6):
    print(animal_number)
    beh_list = glob.glob(('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Behavior_0418/{}/*.csv').format(animal_number))
    sig_list = glob.glob(('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Ref_corrected_0418/{}/*.csv').format(animal_number))
    beh_list.sort()
    sig_list.sort()
    for idx in tqdm(range(len(beh_list))): 
        
        diffrence, Differnece_between_two,change_signal, change_signal_turn, change_behavior, change_behavior_turn=combine_Behavior_signal(beh_list[idx],sig_list[idx])
        if abs(diffrence)> 3:
            if diffrence >0: 
                print(beh_list[idx])
                for_name = beh_list[idx]
                not_working.append(beh_list[idx])
                not_working_sig.append(sig_list[idx])
            elif diffrence < -20:
                print(beh_list[idx])
                for_name = beh_list[idx]
                not_working.append(beh_list[idx])
                not_working_sig.append(sig_list[idx])                

## Run the code that work with the above the code

In [ ]:
for animal_number in range(1,6):
    beh_list = glob.glob(('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Behavior_0418/{}/*.csv').format(animal_number))
    sig_list = glob.glob(('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Ref_corrected_0418/{}/*.csv').format(animal_number))
    beh_list = [x for x in beh_list if x not in not_working]
    sig_list = [x for x in sig_list if x not in not_working_sig]
    beh_list.sort()
    sig_list.sort()
    try:
        os.makedirs('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Combined_Behavioral_Signal_0421/{}'.format(animal_number))
    except FileExistsError:
        # directory already exists
        pass
    for idx in tqdm(range(len(beh_list))): 
        name_for_beh =  beh_list[idx].split('/')[-1]
        diffrence, Differnece_between_two, change_signal, change_behavior, signal, Behavior, Merged_data =combine_Behavior_signal(beh_list[idx],sig_list[idx])
        Merged_data.to_csv('/Users/gunahn/Desktop/MIT/Habit_Breaking_Da_Ast/Data/GFAP_Combined_Behavioral_Signal_0421/{}/sig_combined_{}'.format(animal_number, name_for_beh))